In [1]:
from collections import defaultdict
from gensim.corpora import Dictionary
import spacy
import pandas as pd
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
import time

In [3]:
dataset = pd.read_csv("palabras_corregidas_secundaria.csv", encoding = "latin1")

In [4]:
master_rows = [["porque hace mucho ruido", "porque grita mucho" ],
               ["por que tiene pelo", 'porque tiene mucho pelo y esta redondo'],
               ["que vale mucho", "porque la luna es color plateado"]]

In [5]:
puntajes = [dataset.iloc[:, i].values.tolist() for i in [14,16,18]]

### Todas las respuestas

In [6]:
docs_all = [dataset.iloc[:,i].values.tolist() for i in [13, 15, 17]]
docs_all

[['por que hace mucho ruido',
  'que es un hermano que grita mucho más de lo normal ',
  'que es una persona gritona',
  'porque el niño grita mucho',
  'es muy gritón',
  'que grita todo el tiempo',
  'que su hermano es un gritón',
  'porque su hermanito sube demasiado la voz',
  'se refiere a que el hermano grita y el poeta ve a mundo como pelotas',
  'porque tal vez su hermano grita mucho',
  'por el grita',
  'que no se esta en silencio ',
  'que su hermano puede gritar por que no tiene una pelota de pelos',
  'como esta pequeño da lata',
  'que es bien llorón',
  'cuando al hermano no le compran nada se pone gritar',
  'que su hermano es una pelota que grita',
  'que grita mucho',
  'que habla mucho o grita',
  'que la niña se la pasa llorando o haciendo berrinche ',
  'por que grita mucho',
  'que grita mucho',
  'de que su hermano grita mucho',
  'por que grita',
  'por que no habla en voz baja',
  'por que grita mucho',
  'mi hermanito grita mucho',
  'es gordito y grita mucho'

In [7]:
puntajes = [dataset.iloc[:, i] for i in [14,16,18]]

### Función para limpiar los textos

In [8]:
def docs_to_lists(docs):
    """
    Recibe una lista de strings y regresa
    para cada string una lista limpia
    sin stopwords o puntuaciones para usar
    entrenar modelo.
    """
    temp_nlp = spacy.load("es_core_news_lg")
    doc_corp = []
    for doc in docs:
        clean_doc = []
        for word in temp_nlp(doc):
            if not(word.is_stop or word.is_punct or len(word) == 1):
                clean_doc.append(word.text)
        print(clean_doc)
        doc_corp.append(clean_doc)
    return doc_corp

def doc_to_str(clean_docs):
    """
    Recibe una lista de listas (de la func
    docs_to_lists) y las convierte en un string.
    """
    str_docs = []
    for doc in clean_docs:
        str_docs.append(" ".join(doc))
    return str_docs

def clean_docs(docs):
    """
    Recibe una lista de strings y las limpia de
    puntuaciones y stopwords
    """
    return doc_to_str(docs_to_lists(docs))

###  Limpiamos todas las respuestas y las master rows

In [9]:
texts = [docs_to_lists(lista) for lista in docs_all]

['ruido']
['hermano', 'grita', 'normal']
['persona', 'gritona']
['niño', 'grita']
['gritón']
['grita']
['hermano', 'gritón']
['hermanito', 'sube', 'voz']
['refiere', 'hermano', 'grita', 'poeta', 've', 'mundo', 'pelotas']
['hermano', 'grita']
['grita']
['silencio']
['hermano', 'gritar', 'pelota', 'pelos']
['pequeño', 'lata']
['llorón']
['hermano', 'compran', 'pone', 'gritar']
['hermano', 'pelota', 'grita']
['grita']
['grita']
['niña', 'pasa', 'llorando', 'berrinche']
['grita']
['grita']
['hermano', 'grita']
['grita']
['voz', 'baja']
['grita']
['hermanito', 'grita']
['gordito', 'grita']
['hermano', 'niño', 'pequeño']
['gritos']
['exagera', 'berrinche']
[]
['llora']
['grita']
['divertido']
['tocan', 'grito']
['hermanito', 'energías', 'gritar']
['hermanito', 'gritar']
['hermano', 'grita']
['hermano', 'alguien', 'grita']
['pegan', 'pone', 'gritar']
['pasa', 'gritando', 'pensativo']
['grita']
['hermano', 'gordo', 'pasa', 'gritando']
['frita']
['empieza', 'gritar']
['chilla']
[]
['hermano', '

['hermanos', 'molestar', 'normalmente', 'peleas']
['hermano', 'grita', 'vuelve', 'constante']
['niño', 'llamar', 'atención', 'gritos']
['hermano', 'grita']
['molesto', 'castroso']
['hermanito', 'grita']
['niño', 'gordito', 'pelota', 'grita']
['referencia', 'hermano', 'grita', 'hubieran', 'puesto', 'tipo', 'sobrante', 'pelota']
['gritón']
['altera', 'rápido']
['grita']
['hermanito', 'niño', 'grita']
['hermano', 'grita']
['grita']
['deja', 'gritar']
['simplemente', 'hermano', 'persona', 'gritona', 'pelota', 'refiere', 'cabeza']
['gordo', 'gusta', 'gritar']
['hermano', 'grita']
['niños', 'ruido']
['ruidoso']
[]
['grita', 'rato']
['grita']
['gritando']
['hermano', 'pasa', 'gritando']
['persona', 'regañona', 'grita']
['latoso']
['hermano', 'grita']
['hermanito', 'gritón', 'forma', 'pelota']
['gritando']
['hermano', 'expresivo']
['llora']
['grita']
['anda', 'brincando', 'gritando']
['hermano', 'grita']
['niño', 'grita']
['hermanito', 'grita']
['persona', 'gorda', 'grita']
['refiere', 'herman

['gordo', 'pelos', 'gatos', 'mayoría']
['mascotas', 'pelo']
['suena', 'bonito']
['peina', 'queda', 'cabello', 'bola', 'pelos']
['parecidos']
['viven', 'pájaros']
['desestresas', 'pelota', 'pelos']
['pelos']
['pelos']
['perro', 'pelos']
['pelos']
['bolas', 'sentido']
['pelo']
['pelos']
['pelo']
['chiquito']
['gordo', 'pelos']
['gordo', 'peludo']
['pelos']
['pelo']
['pelo']
['carro', 'pelos']
['cochino']
['seria', 'expresar', 'gato', 'sentido', 'figurado']
['cabeza', 'cabello']
['conejo', 'circular', 'peluda']
['bola', 'pelo']
['personas', 'planeta', 'pelo']
['alguien', 'caerle', 'pelo']
['peludos']
['pelaje']
['pelos']
['mascotas', 'peludas']
['mascotas', 'normalmente', 'peludas']
['tela', 'pelos']
['animalito', 'peludo', 'quedar', 'frase']
['pelos', 'refiere', 'cabellera', 'pelota', 'persona']
[]
['pelos']
['animal', 'peludo']
['forma', 'redonda', 'llena', 'caso', 'cabello']
['pelo']
['pelos']
['fuerte']
['sentido', 'figurado']
['circular', 'pelos']
['pelos', 'estómago']
['mayoría', 'e

['luna', 'blanca', 'plata']
['pelota', 'hierro']
['vale']
['menor', 'fuego', 'importancia']
['alumbra']
['valioso']
['luna', 'blanca', 'plata']
['valioso']
['bala', 'plata']
['refiere', 'luna']
['jarra', 'pelota']
['fortuna']
['pelota', 'dinero']
['mirada', 'plata']
[]
['luna', 'plata']
['luna', 'plata']
['importante']
['pelota', 'plata']
['plata']
['pelota', 'color', 'plata']
['brillando', 'refleja', 'sobresale']
['pelota', 'plata', 'pelota', 'plata']
['hablar', 'canica']
['padre', 'alguien', 'fuerte', 'plata']
['dura']
['luna', 'brilla']
['luna', 'bola', 'brillante']
['collar', 'forma', 'pelota', 'hecha', 'plata']
['luna', 'ligero', 'color', 'plateado']
['luna', 'forma', 'pelota', 'color', 'plata']
['luna', 'color', 'plata', 'gris']
[]
['aluminio']
['luna', 'hecha', 'plata']
['premio']
['luna', 'suena', 'lírico']
[]
['quieres', 'proteger', 'hacerdaño', 'darte']
['luna', 'forma', 'esférica', 'plata']
['pelota', 'color', 'gris']
['valiosa']
['comida', 'vale']
['luna', 'llama']
['vida',

['luna', 'plateada']
['luna', 'redonda', 'color', 'gris', 'brilla']
['luna', 'forma', 'pelota', 'color', 'color', 'plata']
['referencia', 'color', 'luna', 'vemos', 'reflejo', 'sol', 'brillante', 'vista', 'humana', 'lejanía', 'reflejo']
['come', 'refiriendo', 'frase', 'sol', 'luna', 'color', 'plata', 'pienso', 'refiere', 'luna']
['luna', 'redonda', 'sol', 'frase', 'entender', 'sol', 'frase', 'deducir', 'luna', 'luna', 'decirse', 'plateada']
['pelota', 'color', 'plata']
['mente', 'valiosa', 'brillante']
['luna', 'redonda', 'color', 'plata']
['color', 'luna']
['irradia', 'luz', 'blanca', 'reflejo', 'plata']
['luna', 'redonda', 'plateada', 'gris']
['pongo', 'dinero', 'pelota', 'plata']
['metáfora', 'luna', 'bella', 'plata']
['pelota', 'dura', 'plata']
['importante']
['puse', 'vida', 'valioso', 'sacarle', 'provecho']
['pelota', 'dura']
['importancia']
['luna', 'circular', 'color', 'plata']
['hombres', 'mujer', 'cosa', 'linda']
['pelota', 'hecha', 'plata']
['pelota', 'brillante']
['luna', 'c

In [11]:
master_rows = [docs_to_lists(masters) for masters in master_rows]

['ruido']
['grita']
['pelo']
['pelo', 'redondo']
['vale']
['luna', 'color', 'plateado']


### Funciones para entrenar modelo

In [19]:
def modelo(textos,master1, master2):
    frequency = defaultdict(int)
    for text in textos:
        for token in text:
            frequency[token] += 1
    
    texts = [
    [token for token in text if frequency[token] > 1]
    for text in textos
    ]
    
    dictionary = Dictionary(texts)
    
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    lsi1 = LsiModel(corpus, id2word=dictionary, num_topics=2)
    lsi2 = LsiModel(corpus, id2word=dictionary, num_topics=5)
    lsi3 = LsiModel(corpus, id2word=dictionary, num_topics=10)
    lsi4 = LsiModel(corpus, id2word=dictionary, num_topics=20)
    lsi5 = LsiModel(corpus, id2word=dictionary, num_topics=50)
    lsi6 = LsiModel(corpus, id2word=dictionary, num_topics=100)
    lsi7 = LsiModel(corpus, id2word=dictionary, num_topics=150)
    lsi8 = LsiModel(corpus, id2word=dictionary, num_topics=200)\
    
    vec_bow1 = dictionary.doc2bow(master1)
    vec_bow2 = dictionary.doc2bow(master2)
    
    avec_lsi1 = lsi1[vec_bow1]
    avec_lsi2 = lsi2[vec_bow1]
    avec_lsi3 = lsi3[vec_bow1]
    avec_lsi4 = lsi4[vec_bow1]
    avec_lsi5 = lsi5[vec_bow1]
    avec_lsi6 = lsi6[vec_bow1]
    avec_lsi7 = lsi7[vec_bow1]
    avec_lsi8 = lsi8[vec_bow1]
    
    bvec_lsi1 = lsi1[vec_bow2]
    bvec_lsi2 = lsi2[vec_bow2]
    bvec_lsi3 = lsi3[vec_bow2]
    bvec_lsi4 = lsi4[vec_bow2]
    bvec_lsi5 = lsi5[vec_bow2]
    bvec_lsi6 = lsi6[vec_bow2]
    bvec_lsi7 = lsi7[vec_bow2]
    bvec_lsi8 = lsi8[vec_bow2]
    
    index1 = MatrixSimilarity(lsi1[corpus])
    index2 = MatrixSimilarity(lsi2[corpus])
    index3 = MatrixSimilarity(lsi3[corpus])
    index4 = MatrixSimilarity(lsi4[corpus])
    index5 = MatrixSimilarity(lsi5[corpus])
    index6 = MatrixSimilarity(lsi6[corpus])
    index7 = MatrixSimilarity(lsi7[corpus])
    index8 = MatrixSimilarity(lsi8[corpus])
    
    asims1 = index1[avec_lsi1]
    asims2 = index2[avec_lsi2]
    asims3 = index3[avec_lsi3]
    asims4 = index4[avec_lsi4]
    asims5 = index5[avec_lsi5]
    asims6 = index6[avec_lsi6]
    asims7 = index7[avec_lsi7]
    asims8 = index8[avec_lsi8]
    
    bsims1 = index1[bvec_lsi1]
    bsims2 = index2[bvec_lsi2]
    bsims3 = index3[bvec_lsi3]
    bsims4 = index4[bvec_lsi4]
    bsims5 = index5[bvec_lsi5]
    bsims6 = index6[bvec_lsi6]
    bsims7 = index7[bvec_lsi7]
    bsims8 = index8[bvec_lsi8]
    
    sims1 = []
    sims2 = []
    sims3 = []
    sims4 = []
    sims5 = []
    sims6 = []
    sims7 = []
    sims8 = []
    
    for i in range(0,len(asims1)):
        sims1.append(max(asims1[i], bsims1[i]))
        
    for i in range(0,len(asims1)):
        sims2.append(max(asims2[i], bsims2[i]))
        
    for i in range(0,len(asims1)):
        sims3.append(max(asims3[i], bsims3[i]))
        
    for i in range(0,len(asims1)):
        sims4.append(max(asims4[i], bsims4[i]))
        
    for i in range(0,len(asims1)):
        sims5.append(max(asims5[i], bsims5[i]))
    
    for i in range(0,len(asims1)):
        sims6.append(max(asims6[i], bsims6[i]))
        
    for i in range(0,len(asims1)):
        sims7.append(max(asims7[i], bsims7[i]))
        
    for i in range(0,len(asims1)):
        sims8.append(max(asims8[i], bsims8[i]))
    
    
    return [sims1, sims2, sims3, sims4, sims5, sims6, sims7, sims8]
    
def similitudes(lista_textos, master_rows):
    sims = []
    for i in range(0,len(lista_textos)):
        sims.append(modelo(lista_textos[i], master_rows[i][0], master_rows[i][1]))
    return sims

def make_DF(textos, masters):
    start_time = time.time()
    
    sims = similitudes(textos, masters)
    resultados = pd.DataFrame({
        "Explica pelota de gritos" : docs_all[0],
        "a_num_topics_2" : sims[0][0],
        "a_num_topics_5" : sims[0][1],
        "a_num_topics_10" : sims[0][2],
        "a_num_topics_20" : sims[0][3],
        "a_num_topics_50" : sims[0][4],
        "a_num_topics_100" : sims[0][5],
        "a_num_topics_150" : sims[0][6],
        "a_num_topics_200" : sims[0][7],
        "a_puntaje" : puntajes[0],
        "Explica pelota de pelos" : docs_all[1],
        "b_num_topics_2" : sims[1][0],
        "b_num_topics_5" : sims[1][1],
        "b_num_topics_10" : sims[1][2],
        "b_num_topics_20" : sims[1][3],
        "b_num_topics_50" : sims[1][4],
        "b_num_topics_100" : sims[1][5],
        "b_num_topics_150" : sims[1][6],
        "b_num_topics_200" : sims[1][7],
        "b_puntaje" : puntajes[1],
        "Explica pelota de plata" : docs_all[2],
        "c_num_topics_2" : sims[2][0],
        "c_num_topics_5" : sims[2][1],
        "c_num_topics_10" : sims[2][2],
        "c_num_topics_20" : sims[2][3],
        "c_num_topics_50" : sims[2][4],
        "c_num_topics_100" : sims[2][5],
        "c_num_topics_150" : sims[2][6],
        "c_num_topics_200" : sims[2][7],
        "c_puntaje" : puntajes[2]
    })
    
        
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return resultados

In [20]:
resultados = make_DF(texts, master_rows)

--- 2.4527828693389893 seconds ---


In [22]:
from pandasgui import show
show(resultados)

In [21]:
resultados

,Explica pelota de gritos,a_num_topics_2,a_num_topics_5,a_num_topics_10,a_num_topics_20,a_num_topics_50,a_num_topics_100,a_num_topics_150,a_num_topics_200,a_puntaje,...,Explica pelota de plata,c_num_topics_2,c_num_topics_5,c_num_topics_10,c_num_topics_20,c_num_topics_50,c_num_topics_100,c_num_topics_150,c_num_topics_200,c_puntaje
0,por que hace mucho ruido,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,3,...,que es dura o aguanta,0.999199,-0.154721,-0.213425,-0.097355,0.001561,0.000022,9.313226e-10,9.313226e-10,1
1,que es un hermano que grita mucho más de lo no...,0.879225,0.722241,0.711291,0.709738,0.707885,7.071068e-01,7.071068e-01,7.071068e-01,3,...,puede ser una medalla o una esfera color metál...,0.982927,0.733095,0.734544,0.651648,0.391178,0.333648,3.333333e-01,3.333333e-01,1
2,que es una persona gritona,0.939499,0.444658,-0.002068,-0.004309,0.006615,0.000000e+00,0.000000e+00,0.000000e+00,3,...,una joya,0.996429,0.118774,-0.079271,0.010109,0.002751,0.007675,4.656613e-10,4.656613e-10,1
3,porque el niño grita mucho,0.999966,0.750952,0.715416,0.710411,0.708415,7.071068e-01,7.071068e-01,7.071068e-01,3,...,seria muy brillosa la pelota,0.905671,0.092440,-0.024738,-0.003635,-0.001991,-0.000164,1.303852e-08,1.303852e-08,2
4,es muy gritón,0.816204,0.696199,-0.018814,0.016623,-0.000290,3.725290e-09,3.725290e-09,3.725290e-09,3,...,da a conocer que que es uno muy fuerte,0.914773,0.226741,0.682950,0.019615,-0.002500,0.001534,0.000000e+00,0.000000e+00,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,mi hermanito grita mucho,0.997182,0.784445,0.714204,0.708861,0.707976,7.071068e-01,7.071068e-01,7.071068e-01,3,...,pues tal vez la medalla esta llena de plata,0.830043,0.025122,0.026757,0.002368,0.000640,0.001178,0.000000e+00,0.000000e+00,1
466,puede significar que el hermano es ruidoso,0.959746,0.068625,0.102028,0.002397,0.000323,7.450581e-09,7.450581e-09,7.450581e-09,3,...,yo puse que la frase se podría referir a la lu...,0.999977,0.989599,0.890572,0.765933,0.563580,0.518933,5.163978e-01,5.163978e-01,3
467,que grita mucho,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,3,...,que es brillante,0.998975,0.491243,0.191341,0.005050,0.005998,0.001187,1.490116e-08,1.490116e-08,3
468,esta usando la palabra pelota para todo así qu...,0.887852,0.610761,0.388351,-0.002945,0.000285,3.725290e-09,3.725290e-09,3.725290e-09,3,...,porque ya establecimos que a todo le esta dici...,0.863808,0.035469,0.001208,0.000530,0.000494,0.000101,-3.725290e-09,-3.725290e-09,1


In [23]:
resultados.to_csv("resultados_lsi_todos.csv", encoding = "latin1")